In [ ]:
import neat
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import networkx as nx
from reservoirpy import nodes, datasets

import reservoirpy as rpy
rpy.verbosity(0)

# Assuming the helper functions from the provided scripts are imported correctly:
import hierarchical_genome_mutations_operations as hgmo
import evolution_step_functions as esf
import genome_analysis as ga
import genome_transcription as gt
import helper_functions as hf
import hierarchical_genomes as hg

In [ ]:
# Define other required parameters
population_size = 50
n_generations = 200
genome_length = 100
n_timesteps = 2000

# Initialize your dataset
X = datasets.mackey_glass(n_timesteps=n_timesteps, sample_len=2000)
Y = np.roll(X, -1)

# Split the dataset
train_end = int(len(X) * 0.7)
test_start = train_end + 1
X_train, Y_train = X[:train_end], Y[:train_end]
X_test, Y_test = X[test_start:], Y[test_start:]


In [ ]:
# Define your evaluation function using the loaded dataset
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        predictions = np.array([net.activate(xi) for xi in X_train])
        mse = mean_squared_error(Y_train, predictions)
        mae = mean_absolute_error(Y_train, predictions)
        rmse = np.sqrt(mse)
        
        # Combining fitness metrics, you can customize this part
        genome.fitness = -mse  # NEAT works with a maximization problem

# Load the configuration
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, 'config_neat.txt')

# Create the population, which is the top-level object for a NEAT run.
p = neat.Population(config)

# Add a stdout reporter to show progress in the terminal.
p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)
p.add_reporter(neat.Checkpointer(5))

# Run the experiment, pass eval_genomes function and number of generations
winner = p.run(eval_genomes, 50)

# Display the winning genome
print('\nBest genome:\n{!s}'.format(winner))

# Plotting the performance graph
generation = range(len(stats.get_fitness_mean()))
plt.plot(generation, stats.get_fitness_mean(), label="Mean Fitness")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.legend(loc="best")
plt.show()
